In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd



In [3]:
def scrape_ebay_speakers(url, num_pages):
    product_links = [] #scrape all product_links using find_all

    # multiple pages scrape
    for i in range(1, num_pages + 1):
        r = requests.get(f'{url}/sch/i.html?_from=R40&_nkw=speakers&_sacat=0&LH_TitleDesc=0&rt=nc&_pgn={i}')
        soup = BeautifulSoup(r.text, 'html.parser')
        spans = soup.find_all('a', class_="s-item__link")
        for span in spans:
            link = span['href']
            product_links.append(link)

    # scrape inside the product
    product_list = []

    for link in product_links:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'html.parser')

        name_element = soup.find('h1', class_="x-item-title__mainTitle")
        name = name_element.text if name_element else 'None'

        price_element = soup.find('div', class_="x-price-primary")
        price = price_element.text.split() if price_element else []

        seller_element = soup.find('span', class_="ux-textspans ux-textspans--PSEUDOLINK ux-textspans--BOLD")
        seller_information = seller_element.text if seller_element else 'None'

        shipping_charges1 = soup.find('td', class_="ux-table-section__cell")
        shipping_charges_element = shipping_charges1.find('span', class_="ux-textspans") if shipping_charges1 else None
        shipping_charges = shipping_charges_element.text.split() if shipping_charges_element else []

        item_number_element = soup.find('div', class_="ux-layout-section__textual-display ux-layout-section__textual-display--itemId")
        item_number = item_number_element.find('span', class_='ux-textspans ux-textspans--BOLD').text if item_number_element else 'None'

        reviews_element = soup.find('div', class_="vim x-about-this-seller")
        reviews = reviews_element.find('span', class_='ux-textspans ux-textspans--PSEUDOLINK').text if reviews_element else 'None'

        product_data = {
            'Product_Name': name,
            'Price': price[1] if len(price) > 1 else 'None', #if length is not equal to 1 it becomes None
            'Seller': seller_information,
            'Shipping_charges': shipping_charges[1] if len(shipping_charges) > 1 else 'None',
            'Ebay_Product_Number': item_number,
            'Total_Reviews': reviews
        }

        product_list.append(product_data)

    df = pd.DataFrame(product_list)
    df.to_csv("Ebay_Speakers.csv")
    return df

# Example usage
url = 'https://www.ebay.com'
num_pages = 1
data = scrape_ebay_speakers(url, num_pages)
print(data.head(10))

                                        Product_Name      Price  \
0                                               None       None   
1   JBL Flip 6 Portable Bluetooth Portable Speake...  $89.99/ea   
2   2x Pair of Deaf Bonce Apocalypse AP-M61SE 6.5...    $199.80   
3   Bluetooth Portable Speaker,Wireless Bluetooth...     $89.00   
4   Focal Utopia Floorstanding Speakers; JM Labs;...    $114.50   
5   Anker Soundcore Portable Bluetooth Speaker St...  $20.89/ea   
6   JBL Flip 6 Portable Bluetooth Portable Speake...     $50.00   
7   Bluetooth Speaker Wireless Portable Waterproo...  $14.95/ea   
8   Rockville RHB70 Home Theater Compact Powered ...     $49.95   
9   Vizio 2 Satellite Speakers SB3851 3851 Black/...  $19.99/ea   

              Seller Shipping_charges Ebay_Product_Number       Total_Reviews  
0               None             None                None                None  
1        zar-ecomllc           $26.50        235169565294                 807  
2       bigjeffaudio  

In [6]:
data[1:].head(20)

,Product_Name,Price,Seller,Shipping_charges,Ebay_Product_Number,Total_Reviews
1,JBL Flip 6 Portable Bluetooth Portable Speake...,$89.99/ea,zar-ecomllc,$26.50,235169565294,807
2,2x Pair of Deaf Bonce Apocalypse AP-M61SE 6.5...,$199.80,bigjeffaudio,$124.22,165780761764,23116
3,"Bluetooth Portable Speaker,Wireless Bluetooth...",$89.00,hammo_5474,shipping,386094271507,1
4,Focal Utopia Floorstanding Speakers; JM Labs;...,$114.50,pekola_60,$84.95,145276158747,0
5,Anker Soundcore Portable Bluetooth Speaker St...,$20.89/ea,None,$24.81,385138341588,Shop Anker on eBay
6,JBL Flip 6 Portable Bluetooth Portable Speake...,$50.00,liquidation_sales,$36.97,175887317895,118
7,Bluetooth Speaker Wireless Portable Waterproo...,$14.95/ea,rainbow-tech.21,$24.66,264731910144,21322
8,Rockville RHB70 Home Theater Compact Powered ...,$49.95,audiosavings,$74.63,391948561584,764673
9,Vizio 2 Satellite Speakers SB3851 3851 Black/...,$19.99/ea,4surpluscity,$24.79,225691812865,128634
10,Edifier R1280T Powered 2.0 Bookshelf Speakers...,$60.00,cottage_hodgepodge,$121.04,355003580154,1348
